In [ ]:
# reference: 

# skip gram

In [11]:
import sys
sys.path.append('..')  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
from common.trainer import Trainer
from common.optimizer import Adam
from simple_cbow import SimpleCBOW
from common.util import preprocess, create_contexts_target, convert_one_hot
import numpy

In [2]:
text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)

In [66]:
contexts, target = create_contexts_target(corpus, window_size)
target = convert_one_hot(target, vocab_size)
contexts = convert_one_hot(contexts, vocab_size)

# target word는 총 6개(마지막 단어 제외), vocal dict size은 7 dim 
print(target.shape, contexts.shape)

(6, 7) (6, 2, 7)


# 1. fit

In [9]:
window_size = 1
batch_size = 3
max_epoch = 1000

x = target
data_size = len(x)
max_iters = data_size // batch_size
total_loss = 0
loss_count = 0

# 2. first epoch

In [60]:
# shuffle data
t = contexts
idx = numpy.random.permutation(numpy.arange(data_size))
# idx
# >> array([1, 2, 4, 0, 5, 3])

In [ ]:
# shape 변화 없음
x = x[idx]
t = t[idx]

In [86]:
print(x.shape, t.shape)

(6, 7) (6, 2, 7)


In [81]:
def softmax(x):
    if x.ndim == 2:
        x = x - x.max(axis=1, keepdims=True)
        x = np.exp(x)
        x /= x.sum(axis=1, keepdims=True)
    elif x.ndim == 1:
        x = x - np.max(x)
        x = np.exp(x) / np.sum(np.exp(x))
    return x

In [88]:
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
    # 정답 데이터가 원핫 벡터일 경우 정답 레이블 인덱스로 변환
    if t.size == y.size:
        t = t.argmax(axis=1)
             
    batch_size = y.shape[0]

    return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size

In [114]:
temp_t2.argmax(axis=1)

array([2, 3, 4])

In [113]:
cross_entropy_error(y, temp_t2)

1.9460004620410463

# 3. first iter

In [76]:
# batch size = 3
iters = 0

vocab_size = len(word_to_id)
hidden_size = 5
batch_x = x[iters*batch_size:(iters+1)*batch_size]
batch_t = t[iters*batch_size:(iters+1)*batch_size]

# 6개 target 단어에 대한 loss를 구하고 
# 3개(3 words/ per batch)마다 W update
# batch size = 3 -> 1 epoch = 2 iter
# batch_t shape
## 3: 총 6개 dataset 중 batch size=3
## 2: window size=2에 따라 선택된 num of context words
## 7: vocab dict size=7
print(max_iters, batch_x.shape, batch_t.shape)

2 (3, 7) (3, 2, 7)


In [73]:
# 1st iter
# shape: (3, 7)
batch_t[:,0]

array([[0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0],
       [0, 1, 0, 0, 0, 0, 0]], dtype=int32)

In [84]:
# 2nd iter
batch_t[:,1]

array([[0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0]], dtype=int32)

# 4. feed forward

In [155]:
V, H = vocab_size, hidden_size

# loss = model.forward(batch_x, batch_t)
    # h = self.in_layer.forward(target)
    # self.in_layer = MatMul(W_in)
    # w_in = 0.01 * np.random.randn(V, H).astype('f') 
W_in = 0.01 * np.random.randn(V, H).astype('f') 
W_in_grads = [np.zeros_like(W_in)]
h = np.dot(batch_x, W_in)   # h: dense vector
                            # 즉, embbed word
    # s = self.out_layer.forward(h)
    # self.out_layer = MatMul(W_out)
W_out = 0.01 * np.random.randn(H, V).astype('f')
W_out_grads = [np.zeros_like(W_out)]
s = np.dot(h, W_out)     # s: output
                        # 즉, input으로 다시 복원한 벡터

In [156]:
y.shape

(3, 7)

In [110]:
# batch_x의 각 row = input words
## 여기서는 3개 단어
# temp_t1의 각 row = 각 batch_x row에 대응하는 target
## temp_t2까지 고려하면 batch_x row에 대응하는 target은
## window size=1 -> 2개 context 단어
print(batch_x)
print()
print(temp_t1)
print()
print(temp_t2)

[[0 0 0 1 0 0 0]
 [0 0 1 0 0 0 0]
 [0 0 0 0 1 0 0]]

[[1 0 0 0 0 0 0]
 [0 1 0 0 0 0 0]
 [0 0 1 0 0 0 0]]

[[0 0 1 0 0 0 0]
 [0 0 0 1 0 0 0]
 [0 0 0 0 1 0 0]]


In [98]:
# l1 = self.loss_layer1.forward(s, contexts[:, 0])
# self.loss_layer1 = SoftmaxWithLoss()
# softmax(x)
y = softmax(s)  # y는 input word가 복원되었을 때,
                # 각 vocab에 대한 확률

temp_t1 = batch_t[:, 0]
l1 = cross_entropy_error(y, temp_t1)

temp_t2 = batch_t[:, 1]
l2 = cross_entropy_error(y, temp_t2)

loss = l1 + l2

print('y shape', y.shape)
print('loss', loss)

y shape (3, 7)
loss 3.8918379995748777


# 5. backpropagation


In [139]:
# dl1 = self.loss_layer1.backward(dout)
# self.loss_layer1 = SoftmaxWithLoss()
#         backward
#         batch_size = self.t.shape[0]
batch_size = temp_t1.shape[0]
dout = 1

#         dx = self.y.copy()
#         dx[np.arange(batch_size), self.t] -= 1
#         dx *= dout
#         dx = dx / batch_size
dl1 = y.copy()
dl1[np.arange(batch_size), temp_t1.argmax(axis=1)] -= 1 # softmax with loss: y_hat - t
dl1 *= dout
dl1 = dl1 / batch_size

dl2= y.copy()
dl2[np.arange(batch_size), temp_t2.argmax(axis=1)] -= 1
dl2 *= dout
dl2 = dl2 / batch_size

# ds = dl1 + dl2
ds = dl1 + dl2 # 분기 노드이기 때문에 합산해야 함

## 5.1 out layer

In [150]:
# dh = self.out_layer.backward(ds)
# self.out_layer = MatMul(W_out)
dh = np.dot(ds, W_out.T)
dW_out = np.dot(h.T, ds)
W_out_grads[0][...] = dW_out

## 5.2 in layer

In [157]:
# self.in_layer.backward(dh)
dx = np.dot(dh, W_in.T)
dW_in = np.dot(batch_x.T, dh)
W_in_grads[0][...] = dW_in

### 5.3 update W

In [171]:
W_in_grads

[array([[ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
          0.0000000e+00],
        [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
          0.0000000e+00],
        [-4.0502448e-04,  2.9413586e-03,  2.7490219e-03, -1.4102452e-03,
         -5.3677519e-05],
        [ 7.3971311e-03, -2.6854158e-03, -9.5448922e-04, -1.4722566e-04,
          1.2294482e-03],
        [ 1.1642933e-03, -3.4094390e-03,  1.6853473e-03, -5.1500234e-03,
         -4.2240210e-03],
        [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
          0.0000000e+00],
        [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
          0.0000000e+00]], dtype=float32)]

In [170]:
(np.sqrt(h + 1e-7))

array([[0.00031623, 0.00031623, 0.00031623, 0.00031623, 0.00031623],
       [0.00031623, 0.00031623, 0.00031623, 0.00031623, 0.00031623],
       [0.00031623, 0.00031623, 0.00031623, 0.00031623, 0.00031623],
       [0.00031623, 0.00031623, 0.00031623, 0.00031623, 0.00031623],
       [0.00031623, 0.00031623, 0.00031623, 0.00031623, 0.00031623],
       [0.00031623, 0.00031623, 0.00031623, 0.00031623, 0.00031623],
       [0.00031623, 0.00031623, 0.00031623, 0.00031623, 0.00031623]],
      dtype=float32)

In [173]:
# adam
lr = 0.01
h = np.zeros_like(W_in)
W_in -= lr * W_in_grads[0] / (np.sqrt(h + 1e-7))

h = np.zeros_like(W_out)
W_out -= lr * W_out_grads[0] / (np.sqrt(h + 1e-7))